In [2]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize = (10, 6))
np.set_printoptions(precision = 4, suppress = True)

### 계층적 인덱스(MultiIndex) : 축에 대해 다중(둘 이상) 인덱스 단계를 지정할 수 있도록 해줌

In [4]:
data = pd.Series(np.random.randn(9),
                 index = [['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    1.246435
   2    1.007189
   3   -1.296221
b  1    0.274992
   3    0.228913
c  1    1.352917
   2    0.886429
d  2   -2.001637
   3   -0.371843
dtype: float64

In [5]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

- 계층적 인덱스 객체는 **데이터의 부분집합을 부분적 인덱스로 접근** 하는 것이 가능

In [8]:
print(data['b'])
print(data['b':'c'])
print(data.loc[['b', 'd']])
print(data.loc[:, 2])

1    0.274992
3    0.228913
dtype: float64
b  1    0.274992
   3    0.228913
c  1    1.352917
   2    0.886429
dtype: float64
b  1    0.274992
   3    0.228913
d  2   -2.001637
   3   -0.371843
dtype: float64
a    1.007189
c    0.886429
d   -2.001637
dtype: float64


### ```.unstack()``` : 데이터 재배열 (r의 spread())

In [10]:
data.unstack()

,1,2,3
a,1.246435,1.007189,-1.296221
b,0.274992,NaN,0.228913
c,1.352917,0.886429,NaN
d,NaN,-2.001637,-0.371843


### ```.stack()``` : 데이터 재배열 (r의 gather)

In [11]:
data.unstack().stack()

a  1    1.246435
   2    1.007189
   3   -1.296221
b  1    0.274992
   3    0.228913
c  1    1.352917
   2    0.886429
d  2   -2.001637
   3   -0.371843
dtype: float64

In [12]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [13]:
frame.index.names = ['key1', 'key2']
frame

Ohio     Colorado
          Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [14]:
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

```Python
MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']], 
                       names = ['state', 'color'])
```

### 
### 계층의 순서를 바꾸고 정렬
### ```.swaplevel('키1', '키2')``` : 넘겨받은 두 개의 계층 번호나, 이름이 뒤바뀐 새로운 객체를 반환

In [17]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [22]:
frame.sort_index(level = 1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [27]:
frame.swaplevel(0, 1).sort_index(level = 0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 
### 계층별 요약 통계

- key2별 합계

In [30]:
frame.sum(level = 'key2')

C:\Users\이찬솔\AppData\Local\Temp\ipykernel_16152\958648635.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level = 'key2')


state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [32]:
frame.sum(level = 'color', axis = 'columns')

C:\Users\이찬솔\AppData\Local\Temp\ipykernel_16152\3100069771.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level = 'color', axis = 'columns')


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 데이터 프레임의 열 사용

In [33]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


### ```.set_index()``` : 하나 이상의 열을 인덱스로 하는 새로운 데이터 프레임 생성

In [34]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [37]:
# 선택할 열을 제거하지 않음
frame.set_index(['c', 'd'], drop = False) 

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

### ```.reset_index()``` : ```.set_index()```와 반대

In [38]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


### 
### 데이터 합치기

### 데이터베이스 스타일로 데이터 프레임 합치기 (merge, join)

In [39]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})

### ```merge(df1, df2, on, left_on, right_on, how, suffixes)``` : 두 df를 병합

#### merge 함수 인자 목록
|  |  |
| -- | -- |
| left | 병합하려는 데이터 중 좌측 |
| right | 병합하려는 데이터 중 우측 |
| how | join 방법 : ('inner', 'outer', 'left', 'right') |
| on | join하려는 열 이름. 반드시 두 데이터 플에ㅣㅁ 객체 모두에 존재하는 이름이어야 한다. 명시되지 않고 다른 join 키도 주어지지 않으면 left와 right에서 공통되는 열을 join 키로 사용 |
| left_on | join 키로 사용할 좌측 데이터 프레임의 열  |
| right_one | join 키로 사용할 우측 데이터 프레임의 열 |
| left_index | join 키로 사용할 좌측 데이터 프레임의 인덱스 열 (다중 인덱스일 경우 키) |
| right_index | join 키로 사용할 우측 데이터 프레임의 인덱스 열 (다중 인덱스일 경우 키) |
| sort | join 키에 따라 병합된 데이터를 딕셔너리 순으로 정렬. 기본값은 True.|
| suffixes | 열 이름이 겹칠 경우, 각 열 이름 뒤에 붙일 문자열의 튜플. 기본값은 ('\_x', '\_y') |
| copy | False일 경우, 예외적인 경우에 데이터가 결과로 복사되지 않도록 함. 기본값은 항상 복사가 이루어짐 |
| indicator | merge라는 이름의 특별한 열을 추가하여, 각 행의 소스가 어디인지 나타낸다. ('left_only', 'right_only', 'both') |

### 

- **how 옵션에 따른 다양한 join 연산**
|  |  |
| -- | -- |
| 'inner' | 양쪽 테이블 모두에 존재하는 키 조합을 사용 |
| 'left' | 좌측 테이블에 존재하는 모든 키 조합 사용 |
| 'right | 우츨 테이블에 존재하는 모든 키 조합 사용 |
| 'outer' | 양쪽 테이블의 모든 키 조합 사용 |

In [42]:
pd.merge(df1, df2, on = 'key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


- **```left_on```, ```right_on``` : 중복된 열 이름이 없다면 따로 지정**

In [45]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})

In [46]:
pd.merge(df3, df4, left_on = 'lkey', right_on = 'rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


- **```how``` : join의 방향 설정 ('left', 'right', 'outer')**

In [47]:
pd.merge(df1, df2, how = 'outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


- **다대다 병합**

In [49]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})

In [50]:
pd.merge(df1, df2, on = 'key', how = 'left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


#### 다대다 join은 두 행의 데카르트곱을 반환
- df1에는 3개의 'b'행, df2에는 2개의 'b'행 => 결과는 6개의 'b'행

In [51]:
pd.merge(df1, df2, how = 'inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [52]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
pd.merge(left, right, on = ['key1', 'key2'], how = 'outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [55]:
pd.merge(left, right, on = 'key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [56]:
pd.merge(left, right, on = 'key1', suffixes = ('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### 
### 인덱스 병합 : 병합하려는 키가 데이터 프레임의 인덱스일 경우

### => ```left_index = True``` (혹은 ```right_index = True```)


- left1에는 'key'라는 열이 존재, right1에는 'key'라는 열이 없지만, 인덱스가 'key'의 구성과 동일

In [57]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index = ['a', 'b'])

pd.merge(left1, right1, left_on = 'key', right_index = True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [58]:
pd.merge(left1, right1, left_on = 'key', right_index = True, how = 'outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


#### 계측적 인덱스 병합 => 리스트로 여러 개의 열을 지정 (중복되는 인덱스값을 다룰 때는 ```how = 'outer'```)

In [59]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns = ['event1', 'event2'])

pd.merge(lefth, righth, left_on = ['key1', 'key2'], right_index = True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


#### 계측적 인덱스 병합 => 리스트로 여러 개의 열을 지정 (중복되는 인덱스값을 다룰 때는 ```how = 'outer'```)

In [60]:
pd.merge(lefth, righth, left_on = ['key1', 'key2'],
         right_index = True, how = 'outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


#### 양쪽에 공통적으로 존재하는 여러 개의 인덱스 병합

In [61]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index = ['a', 'c', 'e'],
                     columns = ['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index = ['b', 'c', 'd', 'e'],
                      columns = ['Missouri', 'Alabama'])

pd.merge(left2, right2, how = 'outer', left_index = True, right_index = True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


#### 인덱스로 병합 시 ```.join()```메서드 사용
#### ```.join()```메서드는 열이 겹치지 않으며, 완전히 같거나 유사한 인덱스 구조를 가진 여러 개의 데이터 프레임 객체를 병합할 때 사용
- ```.join()```메서드는 left join을 수행

In [67]:
left2.join(right2, how = 'outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


- ```.join()```메서드는 left join을 수행

In [68]:
left1.join(right1, on = 'key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


#### 인덱스 대 인덱스로 두 데이터 프레임을 병합하려면, - ```.join()```메서드 사용

In [69]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index = ['a', 'c', 'e', 'f'],
                       columns = ['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [70]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [66]:
left2.join([right2, another], how = 'outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0
